<a href="https://colab.research.google.com/github/ShanuDey/Vehicle-Number-Plate-Detection-and-OCR-identification/blob/master/Number_Plate_Detection_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# clean
#!rm -rf *
# log
#!cat /var/log/colab-jupyter.log

# Setting up Environment

In [0]:
# clone darkflow repo
!git clone https://github.com/thtrieu/darkflow.git temp
!cd temp && mv * ../
!rm -rf temp
!mkdir -p weights
# downloading weights
!wget https://pjreddie.com/media/files/yolov2.weights -o weights/yolov2.weights
# downloading custom cfg
!wget https://raw.githubusercontent.com/ShanuDey/Custom-yolo-model/master/yolov2-custom.cfg -o cfg/yolov2-custom.cfg
!wget https://raw.githubusercontent.com/ShanuDey/Custom-yolo-model/master/labels.txt
!ls

Cloning into 'temp'...
remote: Enumerating objects: 2706, done.
remote: Total 2706 (delta 0), reused 0 (delta 0), pack-reused 2706
Receiving objects: 100% (2706/2706), 32.98 MiB | 9.67 MiB/s, done.
Resolving deltas: 100% (1760/1760), done.
--2019-09-21 15:28:56--  https://raw.githubusercontent.com/ShanuDey/Custom-yolo-model/master/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13 [text/plain]
Saving to: ‘labels.txt.1’

labels.txt.1        100%[===================>]      13  --.-KB/s    in 0s      

2019-09-21 15:28:57 (2.38 MB/s) - ‘labels.txt.1’ saved [13/13]

cfg	  flow		LICENSE      sample_img  weights
darkflow  labels.txt	preview.png  setup.py	 yolov2-custom.cfg
demo.gif  labels.txt.1	README.md    test	 yolov2.weights


In [0]:
#install darkflow
#!pip install -e .
!python3 setup.py build_ext --inplace

Compiling darkflow/cython_utils/nms.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo2_findboxes.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo_findboxes.pyx because it changed.
[1/3] Cythonizing darkflow/cython_utils/cy_yolo2_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/darkflow/cython_utils/cy_yolo2_findboxes.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/3] Cythonizing darkflow/cython_utils/cy_yolo_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/darkflow/cython_utils/cy_yolo_findboxes.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[3/3] Cythonizing darkflow/cython_utils/nms.pyx
/usr/local/lib/python3.

# Download Dataset
- Data set for this project is downloaded from server
- unzip the download .zip file to get the .json file from it

In [0]:
#download dataset zip file
!wget -O dataset.zip https://cc-uploads.s3.amazonaws.com/open_links/HumAIn+2019/vehicle-number-plate-detection+Datasets.zip

# #unzip the downloaded zip file    
!unzip dataset.zip    

--2019-09-21 15:29:23--  https://cc-uploads.s3.amazonaws.com/open_links/HumAIn+2019/vehicle-number-plate-detection+Datasets.zip
Resolving cc-uploads.s3.amazonaws.com (cc-uploads.s3.amazonaws.com)... 52.216.108.11
Connecting to cc-uploads.s3.amazonaws.com (cc-uploads.s3.amazonaws.com)|52.216.108.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20255 (20K) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]  19.78K   108KB/s    in 0.2s    

2019-09-21 15:29:24 (108 KB/s) - ‘dataset.zip’ saved [20255/20255]

Archive:  dataset.zip
  inflating: Indian_Number_plates.json  


# Import Libraries
- cv2 is opencv library which stands for open source computer vision. Opecv is a collection os algorithms which are used for computer vision.
- numpy is stands for Numerical Python. It is useful library for scitific calcualtion in python. It has multidimentional array object and lots of usefull methods for calculations.
- matplotlib.pyplot is used for ploting images.

In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import tensorflow as tf
from darkflow.net.build import TFNet

# Load data from dataset json file 
- Dataset Indian_Number_plates.json is consisting of nested json object. So I am using pandas json_normalize method to make a dataframe as your requirement

In [0]:
import json
from pandas.io.json import json_normalize

data = list()
with open('Indian_Number_plates.json') as data_file:
    for line in data_file:
        data.append(json.loads(line))

df = json_normalize(data=data,record_path=['annotation'],meta=['content'])

print(df.shape)
df.head()

(245, 6)


,imageHeight,imageWidth,label,notes,points,content
0,466,806,[number_plate],,"[{'x': 0.7220843672456576, 'y': 0.587982832618...",http://com.dataturks.a96-i23.open.s3.amazonaws...
1,449,494,[number_plate],,"[{'x': 0.16194331983805668, 'y': 0.85077951002...",http://com.dataturks.a96-i23.open.s3.amazonaws...
2,314,500,[number_plate],,"[{'x': 0.082, 'y': 0.697452229299363}, {'x': 0...",http://com.dataturks.a96-i23.open.s3.amazonaws...
3,469,500,[number_plate],,"[{'x': 0.434, 'y': 0.6673773987206824}, {'x': ...",http://com.dataturks.a96-i23.open.s3.amazonaws...
4,563,1001,[number_plate],,"[{'x': 0.2087912087912088, 'y': 0.390763765541...",http://com.dataturks.a96-i23.open.s3.amazonaws...


# Download images from url and store it into the test_images directory
- By using the dataframe 'content' columns url, I am downloading the images from server and store them inside the 'test_images' directory with sequential filename
- Filenames are stored inside a list named image_list

In [0]:
#install scikit image
!pip install scikit-image

In [0]:
from skimage import io
import json
from pandas.io.json import json_normalize
import os

data = list()
with open('Indian_Number_plates.json') as data_file:
    for line in data_file:
        data.append(json.loads(line))

df = json_normalize(data=data,record_path=['annotation'],meta=['content'])

print(df.shape)
df.head()


!mkdir -p train
!mkdir -p train/images
!mkdir -p train/annotations

image_list =list()
for i in range(df.shape[0]):
    file_name = str(i)+".jpg"
    file_path = os.getcwd()+"/train/images/"+file_name
    url =  df['content'][i]
    image_list.append(file_name)
    img = io.imread(url)
    cv2.imwrite(file_path,img)
    
    width = df['imageWidth'][i]
    height = df['imageHeight'][i]
    x1 = int(df['points'][i][0]['x']*width)
    y1 = int(df['points'][i][0]['y']*height)
    x2 = int(df['points'][i][1]['x']*width)
    y2 = int(df['points'][i][1]['y']*height)
    
    
    annotation_template = '''<annotation>
  <folder>annotations</folder>
  <filename>'''+file_name+'''</filename>
  <path>'''+file_path+'''</path>
  <source>
      <database>Unknown</database>
  </source>
  <size>
      <width>'''+str(width)+'''</width>
      <height>'''+str(height)+'''</height>
      <depth>3</depth>
  </size>
  <segmented>0</segmented>
  <object>
      <name>Number Plate</name>
      <pose>Unspecified</pose>
      <truncated>0</truncated>
      <difficult>0</difficult>
      <bndbox>
          <xmin>'''+str(x1)+'''</xmin>
          <ymin>'''+str(y1)+'''</ymin>
          <xmax>'''+str(x2)+'''</xmax>
          <ymax>'''+str(y2)+'''</ymax>
      </bndbox>
  </object>
</annotation>'''
    
    f = open('train/annotations/'+str(i)+".xml","w")
    f.write(annotation_template)
    f.close()
    

(245, 6)


# Add downloaded image list to the dataframe
- the image filename list is inserted to the dataframe

In [0]:
df.insert(0,"imageName",image_list)
df.head()

# Training Yolo for Number Plater Detection

In [0]:
#training
config = tf.ConfigProto(log_device_placement=False) 
config.gpu_options.allow_growth=True

with tf.Session(config=config) as sess:
  options = {"model": "./cfg/yolov2-custom.cfg", 
            "load": "./weights/yolov2.weights",
            "batch": 1,
            "epoch": 500,
            "gpu": 0.7,
            "train": True,
            "annotation": "./train/annotations/",
            "dataset": "./train/images/"}
  tfnet = TFNet(options)

tfnet.train()

In [0]:
# # or try this to train
# !python flow --model cfg/yolov2-custom.cfg --load weights/yolov2.weights --train --annotation train/annotations --dataset train/images --gpu 0.8 --epoch 500

In [0]:
# # or download my trained weights
# !mkdir -p ckpt
# !pip install gdown
# !cd ckpt && gdown 'https://drive.google.com/uc?id=1pzv0Ydm7X6RnKEw_ybn6KgPBa7Y1DLhp'
# !cd ckpt && unzip yolov2-14625.zip
# !ls ckpt

#Loading trained custom model

In [0]:

config = tf.ConfigProto(log_device_placement=False)
config.gpu_options.allow_growth=True

with tf.Session(config=config) as sess:
    options ={
        'model':'./cfg/yolov2-custom.cfg',
        'load':14625,
        'threshold':0.6,
        'gpu':1.0
    }
    tfnet = TFNet(options)

# OCR of the license plates
- pytesseract library in python is used for ocr detection here.
- from dataset we get the normalised quadinate of the license plate
- using height and width of the dataset, I calculate the actual points
- cropped image is generated using this corner point
- then this cropped images is gives to the input of the image_to_string method fo pytesseract library
- All the license plate detected result is stored into the license_plates list
- This detected result is them added to dataframe corrosponding to the image

In [0]:
## install tesseract 
!sudo apt-get install tesseract-ocr
!pip install pytesseract

In [0]:
import pytesseract

def getNumberPlateText(img,results):
    number_plates = list()
    for result in results:
        x1 = result['topleft']['x']
        y1 = result['topleft']['y']
        x2 = result['bottomright']['x']
        y2 = result['bottomright']['y']
        cropped_img = img[y1:y2,x1:x2]
        number_plate =  pytesseract.image_to_string(cropped_img)
        number_plate = "".join([i for i in number_plate if i==' ' or i.isalnum()])
        number_plates.append(number_plate)
    return number_plates

In [0]:
number_plates_list = list()
for i in range(df.shape[0]):    
    img = cv2.imread("test_images/"+df['imageName'][i],0)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = tfnet.return_predict(img)
    #print(results)
    img = showBox(img,results)
    number_plates= getNumberPlateText(img,results)
    number_plates_list.append(number_plates)
    print(number_plates)
    #cv2.imshow("test custom model",img)
    #cv2.waitKey(50)

    
df.insert(0,"number_plate_number",number_plates_list)
df[df['number_plate_number'].str.len() > 7].head()

# Finally the result is stored into .csv file

In [0]:
df.to_csv("result.csv")

# Referances
- https://pypi.org/project/pytesseract/
- https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_setup/py_intro/py_intro.html
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.json.json_normalize.html
- https://scikit-image.org/docs/dev/api/skimage.io.html
- https://pjreddie.com/darknet/yolo/